In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
djc_df = pd.read_csv('../data/mira_djc_complete/dim_job_composite.csv')
swf_df = pd.DataFrame()

In [3]:
djc_df.head()

,JOB_NAME,COBALT_JOBID,MACHINE_NAME,QUEUED_TIMESTAMP,QUEUED_DATE_ID,START_TIMESTAMP,START_DATE_ID,END_TIMESTAMP,END_DATE_ID,USERNAME,PROJECT_NAME,COMMAND,QUEUE_NAME,WALLTIME_SECONDS,RUNTIME_SECONDS,NODES_USED,NODES_REQUESTED,CORES_USED,CORES_REQUESTED,LOCATION,EXIT_STATUS,ELIGIBLE_WAIT_SECONDS,ELIGIBLE_WAIT_FACTOR,QUEUED_WAIT_SECONDS,QUEUED_WAIT_FACTOR,REQUESTED_CORE_HOURS,USED_CORE_HOURS,CAPABILITY_USAGE_CORE_HOURS,NONCAPABILITY_USAGE_CORE_HOURS,BUCKETS3_A_USAGE_CORE_HOURS,BUCKETS3_B_USAGE_CORE_HOURS,BUCKETS3_C_USAGE_CORE_HOURS,MACHINE_PARTITION,EXIT_CODE,MODE,RESID,DATA_LOAD_STATUS,CAPABILITY,SIZE_BUCKETS3,PERCENTILE,NUM_TASKS_SUBBLOCK,NUM_TASKS_CONSECUTIVE,NUM_TASKS_MULTILOCATION,NUM_TASKS_SINGLE,COBALT_NUM_TASKS,IS_SINGLE,IS_CONSECUTIVE,IS_MULTILOCATION,IS_SUBBLOCK,IS_SUBBLOCK_ONLY,IS_MULTILOCATION_ONLY,IS_MULTILOCATION_SUBBLOCK,IS_CONSECUTIVE_ONLY,IS_SINGLE_ONLY,IS_NO_TASKS,IS_OTHER,OVERBURN_CORE_HOURS,IS_OVERBURN
0,352220.mira,352220,mira,2014-10-27 03:37:49,20141027,2015-01-24 19:26:14,20150124,2015-01-24 20:52:46,20150124,hsko,LiquidWater,/gpfs/mira-fs0/projects/LiquidWater/hsinyu/ice...,prod-long,43200.0,5192.0,4096.0,4096.0,65536.0,65536.0,MIR-00C00-73FF1-4096,143,1926075,44,7746504,179,94517.4756,94517.4756,0.0,94517.4755,0.0,0.0,94517.4755,MIR-00C00-73FF1-4096,143,script,-1,complete,Not capability,0% <= x < 16.7%,0% <= x < 16.7%,0,0,8,0,8,0,0,1,0,0,1,0,0,0,0,0,0.0,0
1,352221.mira,352221,mira,2014-10-27 03:37:56,20141027,2015-01-24 21:10:34,20150124,2015-01-25 08:44:59,20150125,hsko,LiquidWater,/gpfs/mira-fs0/projects/LiquidWater/hsinyu/ice...,prod-long,43200.0,41665.0,4096.0,4096.0,65536.0,65536.0,MIR-00C00-73FF1-4096,0,1911901,44,7752758,179,758469.9733,758469.9733,0.0,758488.1777,0.0,0.0,758488.1777,MIR-00C00-73FF1-4096,0,script,-1,complete,Not capability,0% <= x < 16.7%,0% <= x < 16.7%,0,0,8,0,8,0,0,1,0,0,1,0,0,0,0,0,0.0,0
2,352222.mira,352222,mira,2014-10-27 03:38:03,20141027,2015-01-25 08:46:13,20150125,2015-01-25 20:18:10,20150125,hsko,LiquidWater,/gpfs/mira-fs0/projects/LiquidWater/hsinyu/ice...,prod-long,43200.0,41517.0,4096.0,4096.0,65536.0,65536.0,MIR-00C00-73FF1-4096,0,1911720,44,7794490,180,755812.1244,755812.1244,0.0,755793.9200,0.0,0.0,755793.9200,MIR-00C00-73FF1-4096,0,script,-1,complete,Not capability,0% <= x < 16.7%,0% <= x < 16.7%,0,0,8,0,8,0,0,1,0,0,1,0,0,0,0,0,0.0,0
3,352223.mira,352223,mira,2014-10-27 03:38:09,20141027,2015-01-25 20:19:09,20150125,2015-01-26 07:53:11,20150126,hsko,LiquidWater,/gpfs/mira-fs0/projects/LiquidWater/hsinyu/ice...,prod-long,43200.0,41642.0,4096.0,4096.0,65536.0,65536.0,MIR-00C00-73FF1-4096,0,1911703,44,7836061,181,758069.4756,758069.4756,0.0,758069.4755,0.0,0.0,758069.4755,MIR-00C00-73FF1-4096,0,script,-1,complete,Not capability,0% <= x < 16.7%,0% <= x < 16.7%,0,0,8,0,8,0,0,1,0,0,1,0,0,0,0,0,0.0,0
4,352224.mira,352224,mira,2014-10-27 03:38:19,20141027,2015-01-27 00:54:35,20150127,2015-01-27 12:28:02,20150127,hsko,LiquidWater,/gpfs/mira-fs0/projects/LiquidWater/hsinyu/ice...,prod-long,43200.0,41607.0,4096.0,4096.0,65536.0,65536.0,MIR-40000-737F1-4096,0,1954889,45,7938976,183,757432.3200,757432.3200,0.0,757432.3200,0.0,0.0,757432.3200,MIR-40000-737F1-4096,0,script,-1,complete,Not capability,0% <= x < 16.7%,0% <= x < 16.7%,0,0,8,0,8,0,0,1,0,0,1,0,0,0,0,0,0.0,0


In [4]:
swf_df['JOB_NUMBER'] = djc_df['COBALT_JOBID'] - djc_df['COBALT_JOBID'].min()
swf_df['SUBMIT_TIME'] = pd.to_datetime(djc_df['QUEUED_TIMESTAMP']).astype(np.int64) // 10.0**9
swf_df['SUBMIT_TIME'] = swf_df['SUBMIT_TIME'] - swf_df['SUBMIT_TIME'].min()
swf_df['WAIT_TIME'] = (pd.to_datetime(djc_df['START_TIMESTAMP']).astype(np.int64) - pd.to_datetime(djc_df['QUEUED_TIMESTAMP']).astype(np.int64)) // 10.0**9
swf_df['RUN_TIME'] = djc_df['RUNTIME_SECONDS']
swf_df['NUMBER_OF_ALLOCATED_PROCESSORS'] = djc_df['CORES_USED']
swf_df['AVERAGE_CPU_TIME_USED'] = -1
swf_df['USED_MEMORY'] = -1
swf_df['REQUESTED_NUMBER_OF_PROCESSORS'] = djc_df['CORES_REQUESTED']
swf_df['REQUESTED_TIME'] = djc_df['WALLTIME_SECONDS']
swf_df['REQUESTED_MEMORY'] = -1
swf_df['STATUS'] = ((djc_df['EXIT_STATUS'] == 0) | (djc_df['EXIT_STATUS'] > 255)).astype(int)
swf_df['USER_ID'] = djc_df.assign(id=(djc_df['USERNAME']).astype('category').cat.codes)['id']
swf_df['GROUP_ID'] = djc_df.assign(id=(djc_df['PROJECT_NAME']).astype('category').cat.codes)['id']
swf_df['EXECUTABLE_NUMBER'] = djc_df.assign(id=(djc_df['COMMAND']).astype('category').cat.codes)['id']
swf_df['QUEUE_NUMBER'] = djc_df.assign(id=(djc_df['QUEUE_NAME']).astype('category').cat.codes)['id']
swf_df['PARTITION_NUMBER'] = djc_df.assign(id=(djc_df['MACHINE_PARTITION']).astype('category').cat.codes)['id']
swf_df['PRECEDING_JOB_NUMBER'] = -1
swf_df['THINK_TIME'] = -1

In [6]:
swf_df.describe()

,JOB_NUMBER,SUBMIT_TIME,WAIT_TIME,RUN_TIME,NUMBER_OF_ALLOCATED_PROCESSORS,AVERAGE_CPU_TIME_USED,USED_MEMORY,REQUESTED_NUMBER_OF_PROCESSORS,REQUESTED_TIME,REQUESTED_MEMORY,STATUS,USER_ID,GROUP_ID,EXECUTABLE_NUMBER,QUEUE_NUMBER,PARTITION_NUMBER,PRECEDING_JOB_NUMBER,THINK_TIME
count,2.763140e+05,2.763140e+05,2.763140e+05,276314.000000,276314.000000,276314.0,276314.0,276314.000000,276314.000000,276314.0,276314.000000,276314.000000,276314.000000,276314.000000,276314.000000,276314.000000,276314.0,276314.0
mean,6.943442e+05,7.060531e+07,1.052359e+05,7349.374773,32851.012804,-1.0,-1.0,32212.590082,11508.103838,-1.0,0.678706,507.166774,229.682828,45103.041945,503.275046,131.925527,-1.0,-1.0
std,5.344312e+05,4.272014e+07,3.590496e+05,11750.518818,68965.316411,0.0,0.0,68912.577552,14334.547058,0.0,0.466974,297.323631,108.623941,22163.491868,40.217722,67.943440,0.0,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,15.000000,0.000000,-1.0,-1.0,16.000000,180.000000,-1.0,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.0,-1.0
25%,2.661080e+05,2.915557e+07,4.840000e+02,425.000000,8192.000000,-1.0,-1.0,8192.000000,2400.000000,-1.0,0.000000,242.000000,166.000000,29622.000000,505.000000,73.000000,-1.0,-1.0
50%,6.205755e+05,6.655491e+07,1.108150e+04,2666.000000,8192.000000,-1.0,-1.0,8192.000000,6000.000000,-1.0,1.000000,523.000000,231.000000,47843.000000,509.000000,136.000000,-1.0,-1.0
75%,1.186245e+06,1.116757e+08,7.250350e+04,9563.000000,32768.000000,-1.0,-1.0,32768.000000,18000.000000,-1.0,1.000000,716.000000,310.000000,64673.000000,509.000000,189.000000,-1.0,-1.0
max,9.963778e+07,1.497250e+08,1.682503e+07,217243.000000,786432.000000,-1.0,-1.0,786432.000000,217200.000000,-1.0,1.000000,1046.000000,504.000000,72332.000000,512.000000,242.000000,-1.0,-1.0
